In [1]:
pip install tensorflow matplotlib pillow numpy


In [6]:
pip install requests


In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import BytesIO
from PIL import Image
from tensorflow.keras.preprocessing import image as k_image

# Function to download image from URL and load it into a tensor
def load_image_from_url(img_url, target_size=(256, 256)):
    try:
        # Custom headers to mimic a browser request
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        # Send GET request to the URL with custom headers
        response = requests.get(img_url, headers=headers)

        # Ensure the URL returns a valid image response
        if response.status_code != 200:
            raise ValueError(f"Failed to download image from {img_url}, status code {response.status_code}")

        # Check the content type to ensure it's an image
        content_type = response.headers.get('Content-Type')
        if 'image' not in content_type:
            raise ValueError(f"URL does not return an image: {img_url}, content type: {content_type}")

        # Load the image from the response content
        img = Image.open(BytesIO(response.content))
        img = img.resize(target_size)  # Resize image
        img = k_image.img_to_array(img)  # Convert to numpy array
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        return tf.convert_to_tensor(img)  # Convert to tensor
    except Exception as e:
        print(f"Error loading image from URL {img_url}: {e}")
        return None

# Display image function
def imshow(img, title=None):
    img = img.numpy()
    img = np.squeeze(img, axis=0)
    img = np.clip(img, 0.0, 255.0).astype('uint8')
    plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()

# Content loss function (MSE between content and generated image)
def content_loss(content, generated):
    return tf.reduce_mean(tf.square(content - generated))

# Style loss function (Gram matrix approach)
def gram_matrix(tensor):
    channels = int(tensor.shape[-1])
    a = tf.reshape(tensor, [-1, channels])
    gram = tf.matmul(a, a, transpose_a=True)
    return gram

def style_loss(style, generated):
    S = gram_matrix(style)
    G = gram_matrix(generated)
    return tf.reduce_mean(tf.square(S - G))

# Total loss function (weighted sum of content and style loss)
def total_loss(content_weight, style_weight, content_image, style_image, generated_image):
    c_loss = content_loss(content_image, generated_image)
    s_loss = style_loss(style_image, generated_image)
    total_loss = content_weight * c_loss + style_weight * s_loss
    return total_loss

# Define a simple function to calculate the gradients for optimization
@tf.function
def compute_gradients(content_weight, style_weight, content_image, style_image, generated_image):
    with tf.GradientTape() as tape:
        tape.watch(generated_image)
        loss = total_loss(content_weight, style_weight, content_image, style_image, generated_image)
    grads = tape.gradient(loss, generated_image)
    return loss, grads

# Function to optimize the image
def optimize_image(content_image, style_image, content_weight=1e3, style_weight=1e-2, num_iterations=100):
    generated_image = tf.Variable(content_image)  # Start with the content image
    optimizer = tf.optimizers.Adam(learning_rate=0.02)

    for i in range(num_iterations):
        loss, grads = compute_gradients(content_weight, style_weight, content_image, style_image, generated_image)
        optimizer.apply_gradients([(grads, generated_image)])

        if i % 10 == 0:
            print(f"Iteration {i}, Loss: {loss.numpy()}")
            imshow(generated_image, title=f"Iteration {i}")

    return generated_image

# Main function to run NST
def run_neural_style_transfer(content_url, style_url, content_weight=1e3, style_weight=1e-2, num_iterations=100):
    # Load content and style images from URLs
    content_image = load_image_from_url(content_url)
    style_image = load_image_from_url(style_url)

    # Check if images are successfully loaded
    if content_image is None or style_image is None:
        print("Error loading images, aborting neural style transfer.")
        return

    # Start the optimization process
    generated_image = optimize_image(content_image, style_image, content_weight, style_weight, num_iterations)

    # Show final output
    imshow(generated_image, title="Final Output")

# Example usage:
content_image_url = 'https://www.w3schools.com/w3images/lights.jpg'  # Photo content image
style_image_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Starry_Night_Over_the_Rhone.jpg/440px-Starry_Night_Over_the_Rhone.jpg'  # Starry Night painting

run_neural_style_transfer(content_image_url, style_image_url, num_iterations=100)


Error loading image from URL https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Starry_Night_Over_the_Rhone.jpg/440px-Starry_Night_Over_the_Rhone.jpg: Failed to download image from https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Starry_Night_Over_the_Rhone.jpg/440px-Starry_Night_Over_the_Rhone.jpg, status code 404
Error loading images, aborting neural style transfer.
